In [2]:
import pandas as pd

# Read in the csv
house_path = 'C:\\Users\\nicho\\OneDrive - The University of Western Ontario\\Ecolux\\Databases\\REFIT\\Regression Training Set\\house5.csv'
house_df = pd.read_csv(house_path)

In [3]:
# Converting the time to int with error handling
def convert_time_to_seconds(time_str):
    try:
        hours, minutes, seconds = [int(part) for part in time_str.split(':')]
        return hours * 3600 + minutes * 60 + seconds
    except ValueError:
        return 0

house_df['Time'] = house_df['Time'].apply(convert_time_to_seconds)

In [12]:
# Creating the x and y inputs
house_df.drop(house_df.iloc[:, 0:1], inplace=True, axis=1)
# selected_features = ['HVAC', 'AlwaysOn', 'Intermit','TimeSin', 'TimeCos', 'DayNumSin', 'DayNumCos', 'MonthSin', 'MonthCos', 'RealTemp', 'ApparTemp', 'Humid']
X = house_df.drop(columns=['HVAC', 'AlwaysOn', 'Intermit'])
 # ['HVAC', 'AlwaysOn', 'Intermit','TimeSin', 'TimeCos', 'DayNumSin', 'DayNumCos', 'MonthSin', 'MonthCos', 'RealTemp', 'ApparTemp', 'Humid']
 #we also need the house number
y = house_df[['HVAC', 'AlwaysOn', 'Intermit']].copy()

# Determining sums
total_energy = X['Total']
y['HVAC_frac'] = y['HVAC'] / total_energy
y['AlwaysOn_frac'] = y['AlwaysOn'] / total_energy
y['Intermit_frac'] = y['Intermit'] / total_energy

# New y with fractions
y_frac = y[['AlwaysOn_frac', 'Intermit_frac', 'HVAC_frac']]
print(y_frac)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import ElasticNet
import numpy as np
import pandas as pd


# Assuming X is your feature matrix and y is the target matrix with 3 columns
X_train, X_test, y_train, y_test = train_test_split(X, y_frac, test_size=0.2, random_state=42)

# y testing data for usage amounts
total_energy_test = np.array(X_test['Total'])
# total_energy_test = X_test['Total']
y_test_actuals = y_test * total_energy_test[:, None]
y_test_actuals_df = pd.DataFrame(y_test_actuals, columns=y_test.columns)


In [14]:
import pandas as pd

# Print the first five rows of y_train
print(y_train.head())

# Check for missing values in y_train
print(y_train.isnull().sum())

# Initialize and fit the model
linear_model = MultiOutputRegressor(ElasticNet(random_state=42, positive=True)).fit(X_train, y_train)

# Predicting the percentages
y_pred_fracs_linear = linear_model.predict(X_test)

# Converting back to energy values
y_pred_actuals_linear = y_pred_fracs_linear * total_energy_test[:, None]


# Definitions:
**MSE**:<br>
-> Definition: MSE measures the average of
the squared differences between actual and predicted values. It is calculated by taking the average of the square of the errors (difference between actual and predicted values). <br>
-> Usage: MSE is useful for evaluating the variance of the model's errors. Lower MSE values indicate better model performance
<br>
**RMSE**:<br>
-> Definition: RMSE is the square root of the mean squared error. It measures the standard deviation of the residuals (errors).<br>
-> Usage: RMSE is more interpretable than MSE because it is in the same units as the target variable. It provides insight into how close the model's predictions are to the actual values.
<br>
**MAE**:<br>
-> Definition: MAE measures the average of the absolute differences between actual and predicted values. It is calculated by taking the mean of the absolute differences between actual and predicted values.<br>
-> Usage: MAE is useful for measuring the average magnitude of errors, without considering their direction. Lower MAE values indicate better model performance.
<br>
**MAPE**:<br>
-> Definition: MAPE measures the average of the absolute percentage differences between actual and predicted values. It expresses errors as percentages of actual values.<br>
-> Usage: MAPE provides insight into the relative size of errors. It is useful for comparing model performance across different scales.
<br>
**R^2**:<br>
-> Definition: R^2 measures the proportion of the variance in the target variable that is explained by the model. It ranges from 0 to 1, where 1 indicates a perfect fit.<br>
-> Usage: R^2 provides a measure of how well the model explains the variability of the target variable. Higher values indicate better model performance.

In [15]:
# Calculate performance metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np

mse = mean_squared_error(y_test_actuals, y_pred_actuals_linear, multioutput='raw_values')
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test_actuals, y_pred_actuals_linear, multioutput='raw_values')
mape = mean_absolute_percentage_error(y_test_actuals, y_pred_actuals_linear, multioutput='raw_values')
mape_percentage = mape * 100
r2 = r2_score(y_test_actuals, y_pred_actuals_linear, multioutput='raw_values')


#when reding the data [0,1,2] The format is [HVAC_frac, AlwaysOn_frac, Intermit_frac] - but are converted back to the value
print("MSE per output:", mse)
print("RMSE per output:", rmse)
print("MAE per output:", mae)
print("MAPE per output:", mape_percentage)#check if this is right
print("R^2 per output:", r2)

# Check if any of the predicted values are negative
negative_values = y_pred_actuals_linear < 0
num_negative_values = np.sum(negative_values)
if num_negative_values > 0:
    print(f"Warning: {num_negative_values} predicted values are negative. Das is baaaad")
else:
    print("No negative predicted values found. Das is go0o0o0od")


MSE per output: [ 10682.51675107 212137.97634034 141786.21964131]
R^2 per output: [-8.3082404  -0.17197638  0.24855303]


In [ ]:
### residuals against predicted values###

import matplotlib.pyplot as plt


# Calculate residuals
residuals_linear = y_test_actuals - y_pred_actuals_linear


# Plot residuals against predicted values
for column in residuals_linear.columns:

    # Extract the predicted values and residuals for the current target variable
    current_predicted_values = y_pred_actuals_linear[:, residuals_linear.columns.get_loc(column)]
    current_residuals = residuals_linear[column]

    # Create a scatter plot of residuals vs. predicted values
    plt.figure(figsize=(8, 6))
    plt.scatter(current_predicted_values, current_residuals)
    plt.axhline(0, color='red', linestyle='--')
    plt.xlabel('Predicted Values')
    plt.ylabel('Residuals')
    plt.title(f'Residuals vs. Predicted Values (Linear Model) - {column}')
    plt.show()


In [ ]:
###QQ plot###

import matplotlib.pyplot as plt
import scipy.stats as stats


# Calculate residuals
residuals_linear = y_test_actuals - y_pred_actuals_linear


# QQ plot of residuals
# Loop through each column in the residuals
for column in residuals_linear.columns:
    # Extract the residuals for the current column
    current_residuals = residuals_linear[column]

    # Create a QQ plot for the current residuals
    plt.figure(figsize=(8, 6))
    stats.probplot(current_residuals, dist="norm", plot=plt)
    plt.title(f'QQ Plot of Residuals (Linear Model) - {column}')
    plt.show()

In [ ]:
# from sklearn.inspection import plot_partial_dependence
# import matplotlib.pyplot as plt

# # Features for which you want to plot PDPs
# features_to_plot = ['Feature1', 'Feature2', 'Feature3']  # Replace with the features you want to visualize

# # Output variables to visualize (index corresponds to the order of columns in y_train)
# target_indices = [0, 1, 2]  # HVAC_frac, AlwaysOn_frac, Intermit_frac

# # Create PDPs
# fig, ax = plt.subplots(figsize=(10, 8))
# plot_partial_dependence(
#     estimator=linear_model,
#     X=X_train,
#     features=features_to_plot,
#     target=target_indices,
#     ax=ax,
#     grid_resolution=50  # Number of points to use in grid
# )

# # Set titles for the plot
# ax[0].set_title('HVAC_frac')
# ax[1].set_title('AlwaysOn_frac')
# ax[2].set_title('Intermit_frac')

# # Show plot
# plt.show()


In [ ]:
# ###SHAP (Shapley Additive Explanations)###

# import shap
# import matplotlib.pyplot as plt


# # Initialize the SHAP explainer for your model
# explainer = shap.Explainer(linear_model, X_train)

# # Compute SHAP values for the test set
# shap_values = explainer.shap_values(X_test)

# # Create SHAP summary plot for each target variable
# for i, target in enumerate(['HVAC', 'AlwaysOn', 'Intermit']):
#     # Use shap.summary_plot for each target variable
#     shap.summary_plot(shap_values[i], X_test, feature_names=selected_features, show=False)
#     plt.title(f'SHAP Summary Plot for {target}')
#     plt.show()

In [ ]:
# Now I'm going to try random forest
from sklearn.ensemble import RandomForestRegressor

# Initialize and fit the model
forest_model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42)).fit(X_train, y_train)

# Predict the fractions
y_pred_fracs_forest = forest_model.predict(X_test)

# Convert predictions back to energy values, as before
y_pred_actuals_forest = y_pred_fracs_forest * total_energy_test[:, None]

In [ ]:
# Calculate performance metrics
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test_actuals, y_pred_actuals_forest, multioutput='raw_values')
r2 = r2_score(y_test_actuals, y_pred_actuals_forest, multioutput='raw_values')

print("MSE per output:", mse)
print("R^2 per output:", r2)

In [10]:
# Now I'm going to try gradient boosting
from xgboost import XGBRegressor

# Initialize and fit the model
boost_model = MultiOutputRegressor(XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)).fit(X_train, y_train)

# Predict the fractions
y_pred_fracs_boost = boost_model.predict(X_test)

# Convert predictions back to energy values, as before
y_pred_actuals_boost = y_pred_fracs_boost * total_energy_test[:, None]

In [11]:
# Calculate performance metrics
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test_actuals, y_pred_actuals_boost, multioutput='raw_values')
r2 = r2_score(y_test_actuals, y_pred_actuals_boost, multioutput='raw_values')

print("MSE per output:", mse)
print("R^2 per output:", r2)

MSE per output: [33623.66344407   994.7341691  33044.27320436]
R^2 per output: [0.82179932 0.13323564 0.81744378]
